In [ ]:
from features import *
import librosa
import librosa.display as display
import scipy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from model_02_functions import *
import sklearn
from train_period_checker import max_pooling
import matplotlib.colors as colors
from timetables import trains_in_period
from wavfile_manipulations import extract_wavfile_and_date
from octave_band import *
from initial_functions import *

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
def expand_singles(array):
    output = []
    for i in array:
        if i==1:
            for j in range(5):
                output.append(1)
        else:
            for j in range(5):
                output.append(0)
    return output

In [ ]:
store = pd.HDFStore('initial_datasets/final1.h5')
data = store['first']

In [ ]:
train = []
for i in range(len(data['cent'])):
    summ = sum([data['cen_eas'][i],data['cen_wes'][i],data['bak_sou'][i],data['bak_nor'][i],data['vic_sou'][i],data['vic_nor'][i]])
    if summ == 0:
        train.append(0)
    else:
        train.append(1)
print(len(train))

In [ ]:
data['train'] = train

In [ ]:
final = pd.concat([data[data.train==1].sample(1477),data[data.train==0].sample(1477)],ignore_index=True).sample(frac=1).reset_index(drop=True)

In [ ]:
X = np.array(final.drop(['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','train'],axis=1))
Y = np.array(final.train)

In [ ]:
X_test = np.array(data.drop(['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','train'],axis=1))
Y_test = np.array(data.train)
print(len(Y_test))

In [ ]:
clf3 = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf3.fit(X, Y)

In [ ]:
labels_svm3 = clf3.predict(X_test[8616:9334])
print(len(labels_svm3))

In [ ]:
data13, time = extract_wavfile_and_date('train_recordings/hour1301.wav', samplerate=3200)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(abs(return_octave_filtered(data13,samplerate=3200,octave_band=125)),1500)
t = np.linspace(0,3600,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,600])
plt.ylim([0,0.0015])
plt.plot(t,train_audio,color='#0098D4',lw=0.3)
"""for i in times:
    for j in times[i]:
        plt.axvline(j,color='#E32017',lw=2.5,alpha=1)
plt.axvline(times['cen_eas'][0],color='#E32017',lw=2.5,alpha=1,label='Expected Train')"""
plt.fill_between(np.linspace(0,3600,3590),[0.0015*i for i in expand_singles(labels_svm3)], color='#E32017',alpha=0.4)
plt.fill_between([1000,2000],[0,0],color='#E32017',alpha=0.4, label='Predicted Train')
plt.legend(loc='upper right')
plt.savefig('final_figures/initial/glm_predict',dpi=200)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [ ]:
principle_components = pd.DataFrame(pca.fit_transform(X_test),columns=['pc1','pc2','pc3'])
principle_components['train'] = Y_test

In [ ]:
print(principle_components)
pca.explained_variance_ratio_

In [ ]:
fig = plt.figure(figsize=(8,4.5))
labels_svm3 = clf3.predict(X_test[8616:9334])
train_audio = np.array(moving_average(principle_components.pc1[8616:9334],10))
t = np.linspace(0,3600,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Principle Component 1')
plt.xlim([0,600])
plt.ylim([0,4])
plt.plot(t,[i for i in train_audio],color='#0098D4',lw=1.5)
"""for i in times:
    for j in times[i]:
        plt.axvline(j,color='#E32017',lw=2.5,alpha=1)
plt.axvline(times['cen_eas'][0],color='#E32017',lw=2.5,alpha=1,label='Expected Train')"""
plt.fill_between(np.linspace(0,3600,3590),[(6*i) for i in expand_singles(labels_svm3)], color='#E32017',alpha=0.4)
plt.fill_between([1000,2000],[0,0],color='#E32017',alpha=0.4, label='Predicted Train')
plt.legend(loc='upper right')
plt.savefig('final_figures/initial/glmpc1',dpi=200)
plt.show()

In [ ]:
labels_svm3 = clf3.predict(X_test[718:1436])
data1, time = extract_wavfile_and_date('train_recordings/hour0101.wav', samplerate=3200)
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(abs(return_octave_filtered(data1,samplerate=3200,octave_band=125)),1500)
t = np.linspace(0,3600,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,600])
plt.ylim([0,0.0015])
plt.plot(t,train_audio,color='#0098D4',lw=0.3)
"""for i in times:
    for j in times[i]:
        plt.axvline(j,color='#E32017',lw=2.5,alpha=1)
plt.axvline(times['cen_eas'][0],color='#E32017',lw=2.5,alpha=1,label='Expected Train')"""
plt.fill_between(np.linspace(0,3600,3590),[0.0015*i for i in expand_singles(labels_svm3)], color='#E32017',alpha=0.4)
plt.fill_between([1000,2000],[0,0],color='#E32017',alpha=0.4, label='Predicted Train')
plt.legend(loc='upper right')
plt.savefig('final_figures/initial/glm_predict2',dpi=200)
plt.show()

In [ ]:
print(len(expand_singles(labels_svm3)))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = np.array(moving_average(principle_components.pc1[718:1436],10))
t = np.linspace(0,3600,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Principle Component 1')
plt.xlim([0,600])
plt.ylim([0,4])
plt.plot(t,[i for i in train_audio],color='#0098D4',lw=1.5)
"""for i in times:
    for j in times[i]:
        plt.axvline(j,color='#E32017',lw=2.5,alpha=1)
plt.axvline(times['cen_eas'][0],color='#E32017',lw=2.5,alpha=1,label='Expected Train')"""
plt.fill_between(np.linspace(0,3600,3590),[6*i for i in expand_singles(labels_svm3)], color='#E32017',alpha=0.4)
plt.fill_between([1000,2000],[0,0],color='#E32017',alpha=0.4, label='Predicted Train')
plt.legend(loc='upper right')
plt.savefig('final_figures/initial/glmpc12',dpi=200)
plt.show()

In [ ]:
data

In [ ]:
new_data = data

In [ ]:
def normalisei(arr):
    """
    scales array to between 0 and 1
    """
    mean = np.mean(np.array(arr).flatten())
    square_mean = np.mean([i**2 for i in np.array(arr).flatten()])
    var = square_mean - mean**2

    for i in range(len(arr)):
        arr[i] = (arr[i]-mean)/(np.sqrt(var))
    
    return arr

In [ ]:
new = []
for column in data.drop(['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','train'],axis=1):
    neww = []
    arr = np.array(data[column])
    for i in np.linspace(0,10052-718,28):
        nor = normalisei(arr[int(i):int(i+359)])
        neww.append(nor)
    new.append(neww)
old = new
for i,j in enumerate(new):
    new[i] = np.concatenate(j)
new = np.transpose(new)

        

In [ ]:
new = pd.DataFrame(new, columns=['zcr','ste','rms','peak','cent','rolloff'])

In [ ]:
new['train'] = train

In [ ]:
final = pd.concat([data[data.train==1].sample(1477),data[data.train==0].sample(1477)],ignore_index=True).sample(frac=1).reset_index(drop=True)
X = np.array(final.drop(['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','train'],axis=1))
Y = np.array(final.train)

In [ ]:
final

In [ ]:
clf3 = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf3.fit(X,Y)

In [ ]:
X_test = new.drop(['train'],axis=1)
labels_svm3 = clf3.predict(X_test[8616:9334])

In [ ]:
print(labels_svm3)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(abs(return_octave_filtered(data13,samplerate=3200,octave_band=125)),1500)
t = np.linspace(0,3600,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,600])
plt.ylim([0,0.0015])
plt.plot(t,train_audio,color='#0098D4',lw=0.3)
"""for i in times:
    for j in times[i]:
        plt.axvline(j,color='#E32017',lw=2.5,alpha=1)
plt.axvline(times['cen_eas'][0],color='#E32017',lw=2.5,alpha=1,label='Expected Train')"""
plt.fill_between(np.linspace(0,3600,3590),[0.0015*i for i in expand_singles(labels_svm3)], color='#E32017',alpha=0.4)
plt.fill_between([1000,2000],[0,0],color='#E32017',alpha=0.4, label='Predicted Train')
plt.legend(loc='upper right')
#plt.savefig('final_figures/initial/glm_predict',dpi=200)
plt.show()

In [ ]:
def find_confusion(pred, act):
    tru_pos = 0
    tru_neg = 0
    fal_pos = 0
    fal_neg = 0
    for i,j in zip(pred,act):
        if i==j:
            if i==1:
                tru_pos+=1
            else:
                tru_neg+=1
        else:
            if i==1:
                fal_pos+=1
            else:
                fal_neg+=1
    return [tru_pos,tru_neg,fal_pos,fal_neg]
    

In [ ]:
def find_metrics(c):
    acc = (c[0]+c[1])/(sum(c))
    pre = c[0]/(c[0]+c[2])
    rec = c[0]/(c[0]+c[3])
    f1 = 2*pre*rec/(pre+rec)
    return [acc,pre,rec,f1]

In [ ]:
clf3 = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf3.fit(X[:2532],Y[:2532])
labels_svm3 = clf3.predict(X[2532:2743])
conf=find_confusion(labels_svm3, Y[2532:2743])
print(find_metrics(conf))